In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

# Define the DenseBlock
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(self._make_dense_layer(in_channels + i * growth_rate, growth_rate))

    def _make_dense_layer(self, in_channels, growth_rate):
        return nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, stride=1, padding=1, bias=False),
        )

    def forward(self, x):
        features = [x]
        for layer in self.layers:
            features.append(layer(torch.cat(features, 1)))
        return torch.cat(features, 1)

# Define the DenseNet
class DenseNet(nn.Module):
    def __init__(self, num_blocks, growth_rate=12, reduction=0.5, num_classes=10):
        super(DenseNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 2 * growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
        self.dense_block1 = DenseBlock(2 * growth_rate, growth_rate, num_blocks[0])
        self.transition1 = self._make_transition(2 * growth_rate + num_blocks[0] * growth_rate, int(reduction * (2 * growth_rate + num_blocks[0] * growth_rate)))

        self.dense_block2 = DenseBlock(int(reduction * (2 * growth_rate + num_blocks[0] * growth_rate)), growth_rate, num_blocks[1])
        self.transition2 = self._make_transition(int(reduction * (2 * growth_rate + num_blocks[0] * growth_rate)) + num_blocks[1] * growth_rate, int(reduction * (int(reduction * (2 * growth_rate + num_blocks[0] * growth_rate)) + num_blocks[1] * growth_rate)))

        self.dense_block3 = DenseBlock(int(reduction * (int(reduction * (2 * growth_rate + num_blocks[0] * growth_rate)) + num_blocks[1] * growth_rate)), growth_rate, num_blocks[2])

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(int(reduction * (int(reduction * (2 * growth_rate + num_blocks[0] * growth_rate)) + num_blocks[1] * growth_rate)) + num_blocks[2] * growth_rate, num_classes)

    def _make_transition(self, in_channels, out_channels):
        return nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.dense_block1(x)
        x = self.transition1(x)
        x = self.dense_block2(x)
        x = self.transition2(x)
        x = self.dense_block3(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Hyperparameters
batch_size = 64
learning_rate = 0.001
epochs = 10

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet(num_blocks=[6, 12, 24], growth_rate=12, reduction=0.5, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    average_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}")

# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified


Epoch 1/10: 100%|██████████| 782/782 [00:59<00:00, 13.18it/s]


Epoch 1/10, Loss: 1.6669


Epoch 2/10: 100%|██████████| 782/782 [00:59<00:00, 13.22it/s]


Epoch 2/10, Loss: 1.2105


...

Epoch 8/10, Loss: 0.6068


Epoch 9/10: 100%|██████████| 782/782 [00:59<00:00, 13.24it/s]


Epoch 9/10, Loss: 0.5660


Epoch 10/10: 100%|██████████| 782/782 [00:59<00:00, 13.25it/s]


Epoch 10/10, Loss: 0.5366
Test Accuracy: 79.28%
